In [ ]:
# Carregamento de dados
import pandas as pd
df= pd.read_csv('../dados/transformed_data.csv', low_memory=False, index_col=[0])

print(";---------------------;")
print("|  Dados Carregados!  |")
print("|---------------------|")
print("|  df: %d         |" %len(df))
print(";---------------------;")

In [ ]:
# criando funçoes
import numpy as np
import re
import nltk
import html
import unidecode

def remove_stopwords(text):
    stopwords = list()
    try: stopwords = nltk.corpus.stopwords.words("portuguese")
    except: nltk.download('stopwords')
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords += ['oi', 'ola', 'opa', 'oie', 'oii', 'oiee', 'oin', 'só', 'so']
    stopwords = [w for w in stopwords if w not in ['mas', 'muito']]
    reg_caracters='\!|\"|\#|\$|\%|\&|\|\'|\(|\)|\*|\+|\,|\.|\/|\n|\:|\;|\<|\=|\>|\?|\@|\[|\\|\]|\^|\_|\`|\{\|\}|\~|[0-9]*'
    remove_acents = lambda sentence: ' '.join([unidecode.unidecode(s) if s.isalnum() else s for s in sentence.split()])

    # remove link do texto
    text = re.sub(r'http\S+', '', text) 
    # adiciona espaço no inicio e final do hex-emoji
    text = text.replace("&#x", " &#x").replace(';', '; ') 
    # transforma o hex-emoji em emoji
    text=html.unescape(text)
    # remove caracteres especiais do texto
    text = re.sub(reg_caracters, "", text)
    # remove acentos das palavras preservando 
    text = remove_acents(text)

    return text

condictions = lambda pol: [pol < -.45, -.45 <= pol < -.2, -.2 <= pol < .2, .2 <= pol < .45, pol > .45]
describes_emojis = {'negativo': "😡", 'irrazoavel': "🙁", 'neutro': "😐", 'razoavel': "🙂", 'positivo': "🤩"}


get_polarity = lambda sentence: sia.polarity_scores(sentence, prin_not_lex=False)[0]["compound"]
get_sentence_size = lambda sentence: len(remove_stopwords(sentence).split())

In [ ]:
# Análise de Sentimentos com custom_vader
import sys
if '../' not in sys.path: sys.path.append('../')
from models.custom_vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer("../dados/custom_vader_lexicon.txt")
print(sia.polarity_scores('eu gostaria ir ao mercado comprar pão'))

In [ ]:
# analisando msgs
test = df[df['from_me']==0].copy()
test = test[['protocolo', 'log_texto', 'from_me', 'emojis']]
test['polarity'] = test['log_texto'].apply(get_polarity)
test['sentence_size'] = test['log_texto'].apply(get_sentence_size)
test['class'] = test["polarity"].apply(lambda p: np.select(condictions(p), [*describes_emojis.keys()]) if p is not None else 'indefinido')

In [ ]:
# # Graphs
# import plotly.graph_objects as go
# from models.graph_tools import Graph

# # fig=Graph(test, "Análise de Polaridade por qtd. de Palavras", is_subplot=True, cols=2, cols_width=[.15,.85], rows=1, row_width=[1])
# fig=Graph(test, "Análise de Polaridade por qtd. de Palavras")
# fig.line('sentence_size', ['polarity'], modes=['markers'], 
#     marker_colorscale='magenta', 
#     hovertemplate='p:%{y:,.2f}%<br>w:%{x}'
# )
# fig.show().update_layout(
#     paper_bgcolor='rgba(230,230,230,230)', 
#     # yaxis={'title': "Polaridade", 'range': [-1, 1]},
#     xaxis={"title":"Qnd.Palavras na Frase"},
#     title_x=.5
# )

# # fig.show().update_layout(yaxis_showticklabels=True, yaxis2_showticklabels=False)